In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import pandas as pd

# Load dataset directly from Google Drive (public access)
file_id = "1un4xuLvenGq7lYbL2ASSa-pjG1FYsTO8"

# This is the correct raw download URL
url = f"https://drive.google.com/uc?id={file_id}&export=download"

df = pd.read_csv(url)
df.head()


,"<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv=""content-type"" content=""text/html; charset=utf-8""/><style nonce=""g2fL68bPGT81Zfvez5Hhyw"">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial",sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption,".uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;text-align:center}.uc-footer a{color:#15c}.uc-footer a:visited{color:#61c}.uc-footer a:active{color:#d14836}.uc-footer-divider{color:#ccc;width:100%}.goog-inline-block{position:relative;display:-moz-inline-box;display:inline-block}* html .goog-inline-block{display:inline}*:first-child+html .goog-inline-block{display:inline}sentinel{}</style><link rel=""icon"" href=""//ssl.gstatic.com/docs/doclist/images/drive_2022q3_32dp.png""/></head><body><div class=""uc-main""><div id=""uc-dl-icon"" class=""image-container""><div class=""drive-sprite-aux-download-file""></div></div><div id=""uc-text""><p class=""uc-warning-caption"">Google Drive can't scan this file for viruses.</p><p class=""uc-warning-subcaption""><span class=""uc-name-size""><a href=""/open?id=1un4xuLvenGq7lYbL2ASSa-pjG1FYsTO8"">allegheny_county_911_EMS_dispatches.csv</a> (334M)</span> is too large for Google to scan for viruses. Would you still like to download this file?</p><form id=""download-form"" action=""https://drive.usercontent.google.com/download"" method=""get""><input type=""submit"" id=""uc-download-link"" class=""goog-inline-block jfk-button jfk-button-action"" value=""Download anyway""/><input type=""hidden"" name=""id"" value=""1un4xuLvenGq7lYbL2ASSa-pjG1FYsTO8""><input type=""hidden"" name=""export"" value=""download""><input type=""hidden"" name=""confirm"" value=""t""><input type=""hidden"" name=""uuid"" value=""52b769fe-76bc-412e-9734-110d876c3ed3""></form></div></div><div class=""uc-footer""><hr class=""uc-footer-divider""></div></body></html>"


## Processing

In [5]:
# storing a dictionary for {priority: description}
# because the same priority levels all have the same desc ... repeat info we don't need
priority_dict = {}
for priority in df.priority.unique():
  priority_desc = df.loc[df.priority == priority, 'priority_desc'].unique()[0]
  priority_dict[priority] = priority_desc

AttributeError: 'DataFrame' object has no attribute 'priority'

In [ ]:
# storing dict that relates city name to city code
city_dict = {}
for city_name in df.city_name.unique():
  city_codes = df.loc[df.city_name == city_name, 'city_code'].unique()
  if len(city_codes) > 0:
    city_dict[city_name] = city_codes[0]

In [ ]:
# dropping columns we don't need... see EDA nb for why
df = df.drop('_id', axis=1)
df = df.drop('call_id_hash', axis=1)
df = df.drop('city_name', axis=1)
df = df.drop('geoid', axis=1)
df = df.drop('service', axis=1)
df = df.drop('priority_desc', axis=1)

In [ ]:
# one hot encoding quarters
quarter_dummies = pd.get_dummies(df['call_quarter'])
df = pd.concat([df, quarter_dummies], axis=1)
df = df.drop('call_quarter', axis=1)

In [ ]:
# ordinal encoding priority levels
from pandas.api.types import CategoricalDtype

order = ['E0', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F4']
priority_series = pd.Series(df.priority.unique())
cat_type = CategoricalDtype(categories=order, ordered=True)
ordered_priority_series = df['priority'].astype(cat_type)


df['priority'] = df['priority'].astype(cat_type);
df.priority.unique()

In [ ]:
# categorizing short description and city code
df['description_short'] = df['description_short'].astype('category')
df['city_code'] = df["city_code"].astype('category')

In [ ]:
# dropping priorities with less than 500 entries
value_counts = df['priority'].value_counts()
df = df[df['priority'].isin(value_counts[value_counts >= 500].index)]

In [ ]:
# were only have ~ 500 missing values so we can just drop them all.
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# checking for missing values
df.isna().sum()

In [ ]:
# tokenizing the description short
description_mapping_dict = {string: idx for idx, string in enumerate(df.description_short.unique())}
print(description_mapping_dict)
df['description_short'] = df['description_short'].map(description_mapping_dict).fillna(-1).astype(int)

In [ ]:
# tokenizing the city code
city_code_mapping_dict = {string: idx for idx, string in enumerate(df.city_code.unique())}
print(city_code_mapping_dict)
df['city_code'] = df['city_code'].map(city_code_mapping_dict).fillna(-1).astype(int)

In [ ]:
# one_hot_encoded = pd.get_dummies(df['description_short'], prefix='description_')
# df = pd.concat([df, one_hot_encoded], axis=1)

# one_hot_encoded = pd.get_dummies(df['city_code'], prefix='description_')
# df = pd.concat([df, one_hot_encoded], axis=1)

# df.drop(columns=['city_code', 'description_short'], inplace=True, axis=1)

## Buidling Model

In [ ]:
# splitting data for regression
from sklearn.model_selection import train_test_split

# Trying without descriptions and city code first
X = df.drop(['priority'], axis=1)
y = df['priority']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [ ]:
# using knn
from sklearn.neighbors import KNeighborsClassifier

num_groups = df.priority.nunique()

knn = KNeighborsClassifier(n_neighbors = num_groups)
knn.fit(X_train, y_train)

In [ ]:
# getting predictions
y_pred = knn.predict(X_test) # if perfect y_pred = y_test

In [ ]:
# analyzing model results
from sklearn.metrics import classification_report, accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# analyzing model results
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 7))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=knn.classes_, yticklabels=knn.classes_)
plt.title("Normalized Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## Defining model building helper functions

In [ ]:
log_file_path = "training_log.txt"
def log_message(message):
    with open(log_file_path, 'a') as log_file:
        log_file.write(f"{message}\n")
    print(message)

In [ ]:
# Lets us cache models so we don't have to rerun all the time
# note, none of the models took more than 2 min so I didn't end up using this
from joblib import load, dump
import traceback

def load_and_test_model(model_path, X_test, y_test, model_name="Model"):
    try:
        model = load(model_path)
        print(f"{model_name} loaded successfully.")

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy for {model_name}: {accuracy:.2f}")

        print(f"Classification Report for {model_name}:")
        print(classification_report(y_test, y_pred))

        cm = confusion_matrix(y_test, y_pred)

        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        plt.figure(figsize=(10, 7))
        sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
        plt.title("Normalized Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show()


    except Exception as e:
        print(f"Error loading/testing {model_name}: {e}")
        print(traceback.format_exc())

### Trying XGBoost Classifier

In [ ]:
# running XGBoost model
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

model_name = 'xgboost'
save_file = f'{model_name}_model.joblib'

try:

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=1, stratify=y_encoded)

    xgb_model = xgb.XGBClassifier(
        objective='multi:softmax',
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=1
    )
    xgb_model.fit(X_train, y_train)
    dump(xgb_model, save_file)
    log_message("XGBoost model trained and saved successfully.")

    load_and_test_model(save_file, X_test, y_test, model_name=model_name)
except Exception as e:
    log_message(f"Error training XGBoost model: {e}")
    log_message(traceback.format_exc())

In [ ]:
# checking feature importance for xgb
model = load('xgboost_model.joblib')
xgb.plot_importance(model)
plt.show()

In [ ]:
# trying RF model ... I rerun the train test split because I like to see it here
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)


model = RandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
model_name = 'Random Forest'
print(f"Accuracy for {model_name}: {accuracy:.2f}")
print(f"Classification Report for {model_name}:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 7))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.title("Normalized Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
importances = model.feature_importances_
feature_names = X.columns if hasattr(X, 'columns') else [f"Feature {i}" for i in range(X.shape[1])]

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 7))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
plt.title("Feature Importance")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.tight_layout()
plt.show()

## Trying Unsupervised Learning

In [ ]:
# trying kmeans model
# scaling data
# dropping priority because thats the baseline we are trying to beat and replace
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

X = df.drop('priority', axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# doign pca for elbow curve
from sklearn.decomposition import PCA
pca = PCA(n_components=9)
X_pca = pca.fit_transform(X_scaled)

pca.explained_variance_ratio_

In [ ]:
#elbow curve
pca_full = PCA().fit(X_scaled)
cumulative_variance = pca_full.explained_variance_ratio_.cumsum()

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.title('Elbow Curve for PCA')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid()
plt.show()

## KMeans

We found that there are 4 well defined clusters when using 3 dimensions.

2D:

In [ ]:
# running 2D kmeans with 4 clusters

n_components = 2
n_clusters = 4

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans.fit(X_pca)
# dump(kmeans, save_file)
log_message("KMeans model trained and saved successfully.")

predicted_clusters = kmeans.labels_

In [ ]:
# plotting
from matplotlib.colors import ListedColormap

predicted_clusters = kmeans.labels_

unique_clusters = np.unique(predicted_clusters)
cluster_colors = ListedColormap(plt.cm.tab10.colors[:len(unique_clusters)])

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=predicted_clusters, cmap=cluster_colors, s=30, alpha=0.7)

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cluster_colors(i), markersize=10)
            for i in range(len(unique_clusters))]
plt.legend(handles, [f'Cluster {i}' for i in unique_clusters], title="Clusters", loc="upper right")

plt.title("KMeans Clusters (2D PCA)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

3D:

In [ ]:
# running 3D kmeans with 4 clusters

n_components = 3
n_clusters = 4

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans.fit(X_pca)
log_message("KMeans model trained and saved successfully.")

predicted_clusters = kmeans.labels_

In [ ]:
# plotting
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter_3d = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=predicted_clusters, cmap='viridis', s=30, alpha=0.7)
fig.colorbar(scatter_3d, label='Cluster Label')
ax.set_title("KMeans Clusters (3D PCA)")
ax.set_xlabel("Principal Component 1")
ax.set_ylabel("Principal Component 2")
ax.set_zlabel("Principal Component 3")
plt.show()

In [ ]:
# plottign in 3d with plotly
import plotly.express as px

num_points_to_show = 10000

df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = predicted_clusters

df_pca['Cluster'] = df_pca['Cluster'].astype(str)

fig = px.scatter_3d(
    df_pca.iloc[np.linspace(0, len(X_pca) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))
# fig.update_layout(
#     scene=dict(
#         xaxis_title="Principal Component 1",
#         yaxis_title="Principal Component 2",
#         zaxis_title="Principal Component 3"
#     )
# )


fig.update_layout(
    scene=dict(
        xaxis_title="",
        yaxis_title="",
        zaxis_title=""
    )
)

fig.show()

In [ ]:
# plottign in 3d with plotly

# this is to save as hmtl and make gif which im doing on local
import plotly.express as px

num_points_to_show = 10000

df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = predicted_clusters

df_pca['Cluster'] = df_pca['Cluster'].astype(str)

fig = px.scatter_3d(
    df_pca.iloc[np.linspace(0, len(X_pca) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))

fig.update_layout(
    title=None,  # Remove title
    showlegend=False,  # Remove legend
    scene=dict(
        xaxis=dict(title="", showticklabels=False),  # Remove x-axis label and tick numbers
        yaxis=dict(title="", showticklabels=False),  # Remove y-axis label and tick numbers
        zaxis=dict(title="", showticklabels=False),  # Remove z-axis label and tick numbers
    )
)

fig.show()

fig.write_html("3dplot.html", include_plotlyjs=True, full_html=True, div_id="plot")

Using 8 clusters:

### 2D

In [ ]:
# running 2D kmeans with 8 clusters

n_components = 2
n_clusters = 8

pca_8clust = PCA(n_components=n_components)
X_pca_8clust = pca.fit_transform(X_scaled)

kmeans_8clust = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans_8clust.fit(X_pca)
# dump(kmeans, save_file)
log_message("KMeans model trained and saved successfully.")

predicted_clusters_8clust = kmeans.labels_

In [ ]:
# plotting
from matplotlib.colors import ListedColormap

unique_clusters_8clust = np.unique(predicted_clusters_8clust)
cluster_colors_8clust = ListedColormap(plt.cm.tab10.colors[:len(unique_clusters_8clust)])

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca_8clust[:, 0], X_pca_8clust[:, 1], c=predicted_clusters_8clust, cmap=cluster_colors_8clust, s=30, alpha=0.7)

handles_8clust = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cluster_colors(i), markersize=10)
            for i in range(len(unique_clusters_8clust))]
plt.legend(handles_8clust, [f'Cluster {i}' for i in unique_clusters_8clust], title="Clusters", loc="upper right")

plt.title("KMeans Clusters (2D PCA)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

## 3D

In [ ]:
# running 3D kmeans with 8 clusters

n_components = 3
n_clusters = 8

pca_8clust = PCA(n_components=n_components)
X_pca_8clust = pca.fit_transform(X_scaled)

kmeans_8clust = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans_8clust.fit(X_pca)
log_message("KMeans model trained and saved successfully.")

predicted_clusters_8clust = kmeans_8clust.labels_

KMeans Silhouette Score: 0.36

In [ ]:
# plots
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter_3d = ax.scatter(X_pca_8clust[:, 0], X_pca_8clust[:, 1], X_pca_8clust[:, 2], c=predicted_clusters_8clust, cmap='viridis', s=30, alpha=0.7)
fig.colorbar(scatter_3d, label='Cluster Label')
ax.set_title("KMeans Clusters (3D PCA)")
ax.set_xlabel("Principal Component 1")
ax.set_ylabel("Principal Component 2")
ax.set_zlabel("Principal Component 3")
plt.show()

In [ ]:
# 3d plotting
num_points_to_show = 10000

df_pca_8clust = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca_8clust['Cluster (8)'] = predicted_clusters_8clust

df_pca_8clust['Cluster (8)'] = df_pca_8clust['Cluster (8)'].astype(str)

fig = px.scatter_3d(
    df_pca_8clust.iloc[np.linspace(0, len(X_pca_8clust) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster (8)',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster (8)': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3"
    )
)

fig.show()

fig.write_html("3dplot.html", include_plotlyjs=True, full_html=True, div_id="plot")

## Gaussian Mixture Model

In [ ]:
# ran GMM
# didn't provide any extra information
from sklearn.mixture import GaussianMixture
n_components = 3

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

gmm = GaussianMixture(n_components=n_components, random_state=1)
gmm.fit(X_scaled)

pred_lables = gmm.predict(X_scaled)

### Taking a closer look at the new classes (with just 4 classes)

In [ ]:
# adding the newfound cluster to the original data set
df["Cluster"] = df_pca['Cluster']

In [ ]:
# df[df.Cluster == 1].head()

In [ ]:
df_0.head(8)

In [ ]:
df_1.head(8)

In [ ]:
df_2.head(8)

In [ ]:
df_3.head(8)

it looks like the model just seperated them by Quarter... lets try to remove that and recluster

## Clustering without quarter

In [ ]:
# gonna run new model without quarters...
X = df.drop(['priority', 'Q1', 'Q2', 'Q3', 'Q4', 'Cluster'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# running 3D kmeans with 4 clusters

n_components = 3
n_clusters = 4

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans.fit(X_pca)
log_message("KMeans model trained and saved successfully.")

predicted_clusters = kmeans.labels_

In [ ]:
# plotting
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
scatter_3d = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=predicted_clusters, cmap='viridis', s=30, alpha=0.7)
fig.colorbar(scatter_3d, label='Cluster Label')
ax.set_title("KMeans Clusters (3D PCA)")
ax.set_xlabel("Principal Component 1")
ax.set_ylabel("Principal Component 2")
ax.set_zlabel("Principal Component 3")
plt.show()

In [ ]:
# 3d plotting with plotly

num_points_to_show = 10000

df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = predicted_clusters

df_pca['Cluster'] = df_pca['Cluster'].astype(str)

fig = px.scatter_3d(
    df_pca.iloc[np.linspace(0, len(X_pca) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3"
    )
)

fig.show()

fig.write_html("3dplot.html", include_plotlyjs=True, full_html=True, div_id="plot")

In [ ]:
# running 3D kmeans with 8 clusters and plotting in 3d

n_components = 3
n_clusters = 8

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans.fit(X_pca)
log_message("KMeans model trained and saved successfully.")

predicted_clusters = kmeans.labels_

num_points_to_show = 10000

df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = predicted_clusters

df_pca['Cluster'] = df_pca['Cluster'].astype(str)

fig = px.scatter_3d(
    df_pca.iloc[np.linspace(0, len(X_pca) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3"
    )
)

fig.show()

In [ ]:
# running 3D kmeans with 6 clusters and plotting in 3d

n_components = 3
n_clusters = 6

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans.fit(X_pca)
log_message("KMeans model trained and saved successfully.")

predicted_clusters = kmeans.labels_

num_points_to_show = 10000

df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = predicted_clusters

df_pca['Cluster'] = df_pca['Cluster'].astype(str)

fig = px.scatter_3d(
    df_pca.iloc[np.linspace(0, len(X_pca) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3"
    )
)

fig.show()

In [ ]:
## Tring with dif number of clusters.... we found 3 had the best results

In [ ]:
# running 3D kmeans with 3 clusters and plotting in 3d

n_components = 3
n_clusters = 3

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init='auto')
kmeans.fit(X_pca)
log_message("KMeans model trained and saved successfully.")

predicted_clusters = kmeans.labels_

num_points_to_show = 10000

df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = predicted_clusters

df_pca['Cluster'] = df_pca['Cluster'].astype(str)

fig = px.scatter_3d(
    df_pca.iloc[np.linspace(0, len(X_pca) - 1, num_points_to_show, dtype=int)],
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',
    title="3D Visualization of KMeans Clusters (PCA)",
    labels={'Cluster': 'Cluster Label'},
    opacity=0.8
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(
    scene=dict(
        xaxis_title="Principal Component 1",
        yaxis_title="Principal Component 2",
        zaxis_title="Principal Component 3"
    )
)

fig.show()

In [ ]:
# adding the cluster from the 3D, 3 cluster model to the original df
df["Cluster"] = df_pca['Cluster']

In [ ]:
df.Cluster.nunique()

In [ ]:
# grouping our data points by cluster
grouped = df.groupby('Cluster')

In [ ]:
grouped.describe()

In [ ]:
# plotting to try to see how each cluster is different
sns.boxplot(data=df, x='Cluster', y='call_year')
plt.title('Call Year Distribution by Cluster')
plt.show()

sns.countplot(data=df, x='priority', hue='Cluster')
plt.title('Priority Counts by Cluster')
plt.show()


In [ ]:
# trying to find statistical significance

from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df['priority'], df['Cluster'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-Square Test p-value for `priority`: {p}")

In [ ]:
# trying to find statistical significance

from scipy.stats import f_oneway

anova_result = f_oneway(*[group['call_year'] for name, group in grouped])
print(f"ANOVA Test p-value for `call_year`: {anova_result.pvalue}")

In [ ]:
# plotting to try to see how each cluster is different

sns.violinplot(data=df, x='Cluster', y='call_year', inner='quartile')
plt.title('Call Year Distribution by Cluster')
plt.show()

In [ ]:
# plotting to try to see how each cluster is different

g = sns.FacetGrid(df, col='Cluster', sharex=True, sharey=True, height=4)
g.map(plt.hist, 'call_year', bins=10, color='skyblue', edgecolor='black')
g.set_axis_labels('Call Year', 'Frequency')
g.set_titles('Cluster {col_name}')
plt.show()